In [1]:
import os
os.chdir('../')

import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd
import numpy as np

from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from nltk.corpus import stopwords
import nltk

from dataloader import load_dataset
from gan_helper import get_cycle_gan_network, get_criterions, get_optimizers, get_schedulers
from config import *
from utils import load_model
from train_helper import evaluate_cycle_gan
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from tqdm.auto import tqdm
from utils import get_sentence_from_tensor

/home/jovyan/.local/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)
/home/jovyan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jovyan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jovyan/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a

generate config dictionary...


In [2]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
DEVICE = 'cuda:4' if torch.cuda.is_available() else 'cpu'
BATCH_SIZE = 128

#### Load Dataset

In [4]:
source, iterators = load_dataset(BATCH_SIZE, DEVICE, mode='inference')
train_iterator, dev_iterator, test_iterator = iterators

#### Prepare Network

In [ ]:
G_INPUT_DIM = len(source.vocab)
G_OUTPUT_DIM = len(source.vocab)
SOS_IDX = source.vocab.stoi['<sos>']
PAD_IDX = source.vocab.stoi['<pad>']

In [6]:
criterion_g_ab, criterion_g_ba, criterion_gan, \
    criterion_discriminator, criterion_cycle, \
    criterion_identity = get_criterions(PAD_IDX, DEVICE)

In [7]:
g_ab, g_ba, d_a, d_b = get_cycle_gan_network(G_INPUT_DIM, 
                                             G_OUTPUT_DIM, 
                                             DEVICE, 
                                             PAD_IDX, 
                                             SOS_IDX,
                                             True, 
                                             True)

load pretrained generator...
load pretrained discriminator...


In [8]:
classifier_ckpt = 's-nlp/roberta_toxicity_classifier' # 'roberta-base'
classifier = AutoModelForSequenceClassification.from_pretrained(classifier_ckpt)
tokenizer = AutoTokenizer.from_pretrained(classifier_ckpt)
# load_model(model=classifier, path='../classifier/model/ft-robertoxic-classifier.pth')
classifier = classifier.to(DEVICE)

Some weights of the model checkpoint at s-nlp/roberta_toxicity_classifier were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


#### Inference

In [9]:
loss_gan_ab, loss_gan_ba, bleu_score_a, bleu_score_b, sta_ab, sta_ba = evaluate_cycle_gan(
    source,
    DEVICE,
    g_ab,
    g_ba,
    d_a,
    d_b,
    iterators[2], # Test data
    criterion_gan,
    sta_tokenizer=tokenizer,
    sta_model=classifier
)
print(f'\nloss_gan_ab: {loss_gan_ab} | loss_gan_ba: {loss_gan_ba} | bleu_score_a: {bleu_score_a} | bleu_score_b: {bleu_score_b}')

  0%|          | 0/79 [00:00<?, ?it/s]


loss_gan_ab: 0.6930230182560184 | loss_gan_ba: 0.7931372904324834 | bleu_score_a: 92.17068387589649 | bleu_score_b: 95.20920725869465


In [10]:
def predict_label(text):
    tokenized = tokenizer(text,
                         truncation=True,
                         max_length=128,
                         padding='max_length',
                         return_tensors='pt')
    tokenized = {k: v.to(DEVICE) for k, v in tokenized.items()}
    with torch.no_grad():
        out = classifier(**tokenized).logits
    proba = torch.softmax(out, dim=1).squeeze()
    label = torch.argmax(proba)
    
    torch.cuda.empty_cache()
    del tokenized
    
    return {'normal_proba': proba[0].item(),
            'toxic_proba': proba[1].item(),
            'predicted_label': label.item()}

In [11]:
res_ab = []
res_ba = []

d_a.eval()
d_b.eval()
smoother = SmoothingFunction()

with torch.no_grad():
    with tqdm(total=len(test_iterator)) as pbar:
        for i, batch in enumerate(test_iterator):
            pbar.update(1)
            
            real_a = batch.src.to(DEVICE)
            real_b = batch.trg.to(DEVICE)
            
            _, fake_b = g_ab(real_a, 0)
            _, fake_a = g_ba(real_b, 0)
            
            # Save A to B scores     
            real_a_sentences = get_sentence_from_tensor(source, real_a)
            fake_b_sentences = get_sentence_from_tensor(source, fake_b)
            for real_a_sentence, fake_b_sentence in zip(real_a_sentences, fake_b_sentences):
                bleu_score = sentence_bleu([real_a_sentence], fake_b_sentence, smoothing_function=smoother.method4) * 100
                sta_score = predict_label(' '.join(fake_b_sentence))
                res_ab.append({"source": " ".join(real_a_sentence), 
                               "gen": " ".join(fake_b_sentence), 
                               "Bleu Score": bleu_score,
                               "Normal Probability": sta_score['normal_proba'],
                               "Toxic Probability": sta_score['toxic_proba'],
                               "Predicted Label": sta_score['predicted_label']})

            # Save B to A scores   
            real_b_sentences = get_sentence_from_tensor(source, real_b)
            fake_a_sentences = get_sentence_from_tensor(source, fake_a)
            for real_b_sentence, fake_a_sentence in zip(real_b_sentences, fake_a_sentences):
                bleu_score = sentence_bleu([real_b_sentence], fake_a_sentence, smoothing_function=smoother.method4) * 100
                sta_score = predict_label(' '.join(fake_a_sentence))
                res_ba.append({"source": " ".join(real_b_sentence), 
                               "gen": " ".join(fake_a_sentence), 
                               "Bleu Score": bleu_score,
                               "Normal Probability": sta_score['normal_proba'],
                               "Toxic Probability": sta_score['toxic_proba'],
                               "Predicted Label": sta_score['predicted_label']})


  0%|          | 0/79 [00:00<?, ?it/s]

In [12]:
df_ab = pd.DataFrame(res_ab)
df_ab["Original Length"] = df_ab["source"].apply(lambda x: len(x.split()))
# df_ab_bleu_by_length = pd.DataFrame(df_ab.groupby(['Original Length'])['Bleu Score'].mean())
# df_ab_bleu_by_length = df_ab_bleu_by_length.reset_index()
# df_ab_bleu_by_length["network_type"] = "G_AB"
df_ab.head(20)

,source,gen,Bleu Score,Normal Probability,Toxic Probability,Predicted Label,Original Length
0,stop censoring me you douche <unk> .,stop censoring me you renounced <unk> .,48.892302,0.996619,0.003381,0,7
1,i think wikipedia is a crap .,i think wikipedia is a crap .,100.000000,0.010503,0.989497,1,7
2,"so , haha u losers lol !","so , haha u losers lol !",100.000000,0.009367,0.990633,1,7
3,wake up your ignorance is dangerous .,wake up your ignorance is dangerous .,100.000000,0.006464,0.993536,1,7
4,trump just employs racists and bigots .,trump just employs racists and bigots .,100.000000,0.005610,0.994389,1,7
5,: : mooch ass grassy ass .,: : mooch ass grassy ass .,100.000000,0.004662,0.995338,1,7
6,"what s up prissy nigga , .","what s up yahoo potheads , .",33.652250,0.978782,0.021218,0,7
7,throw him down the nasty well .,throw him down the nasty well .,100.000000,0.003944,0.996056,1,7
8,what utter hypocrisy and bs ! .,what utter hypocrisy and bs ! .,100.000000,0.003858,0.996142,1,7
9,so you know his penis length ?,so you know his penis length ?,100.000000,0.003231,0.996769,1,7


In [13]:
df_ab[(df_ab['Bleu Score'] < 50) & \
      (df_ab['Predicted Label'] == 0) & \
      (~df_ab["source"].str.contains('<unk>')) & \
      (~df_ab["gen"].str.contains('<unk>'))].sort_values(by=['Bleu Score'], ascending=False).head(20)

,source,gen,Bleu Score,Normal Probability,Toxic Probability,Predicted Label,Original Length
213,who cares what this dirtbag thinks !,who cares what this taxs thinks !,48.892302,0.997833,0.002167,0,7
641,"hold your goddamn horses , already .","hold your avatar horses , already .",48.892302,0.998929,0.001071,0,7
3966,be an asshole about it though ),be an alleging about it though ),48.892302,0.999899,0.000101,0,7
978,do nt fck with my friends .,do nt laughed with my friends .,48.892302,0.999947,0.000053,0,7
507,sovereign state my fat hairy nuts .,sovereign state my fat hydroelectric nuts .,48.892302,0.987328,0.012672,0,7
4305,anyone other than a smelly liberal .,anyone other than a peice liberal .,48.892302,0.998125,0.001875,0,7
4009,what da fucks a gigi ? !,what da venezuelas a gigi ? !,48.892302,0.997184,0.002816,0,7
3599,what a twat that fella is .,what a faceless that fella is .,48.892302,0.995012,0.004988,0,7
3595,will wright is a fuckin genius !,will wright is a wahine genius !,48.892302,0.992874,0.007126,0,7
3593,her moms boobs are better though .,her moms foes are better though .,48.892302,0.996664,0.003336,0,7


In [14]:
from sklearn.metrics import accuracy_score
import numpy as np


print(f"STA: {accuracy_score(np.zeros(len(df_ab), dtype=int), df_ab['Predicted Label'].to_numpy())}")
print(f"BLEU: {df_ab['Bleu Score'].mean()}")

STA: 0.1089
BLEU: 92.19621573635399


In [ ]:
df_ab[['source', 'gen']].to_csv('../data/seq2seq/output/detoxified-2.txt', sep='\t', index=False, header=True)